In [1]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import time

In [2]:
# Проверка доступности GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

torch.manual_seed(42)

writer = SummaryWriter()

batch_size = 64  # Сделаем размер батча конфигурируемым

train_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=True,
    download=True,
    transform=torchvision.transforms.ToTensor()
)
test_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=False,
    download=True,
    transform=torchvision.transforms.ToTensor()
)

train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)
test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=True
)

Using device: cpu
Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 50.5MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 1.92MB/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:00<00:00, 12.9MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 8.75MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [3]:
class MNISTPerceptron(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = torch.nn.Linear(784, 200)
        self.linear2 = torch.nn.Linear(200, 10)

    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.linear1(x)
        x = torch.nn.functional.relu(x)
        x = self.linear2(x)
        return torch.nn.functional.softmax(x, dim=1)

def loss_fn(y_pred, y_true):
    return torch.nn.functional.cross_entropy(y_pred, y_true)

model = MNISTPerceptron().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 5
best_accuracy = 0

In [8]:
# Замер времени
for epoch in range(epochs):
    model.train()
    if device.type == 'cuda':
        torch.cuda.synchronize()  # Синхронизация перед началом замера времени
    epoch_start_time = time.time()

    error = 0.0
    for x, y in train_dataloader:
        x, y = x.to(device), y.to(device)  # Перенос данных на GPU
        optimizer.zero_grad()
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        loss.backward()
        optimizer.step()

        error += loss.item()

    if device.type == 'cuda':
        torch.cuda.synchronize()  # Синхронизация перед окончанием замера времени
    epoch_end_time = time.time()
    epoch_duration = epoch_end_time - epoch_start_time
    print(f'Epoch {epoch + 1}/{epochs}, Loss: {error / len(train_dataloader):.4f}, Time: {epoch_duration:.2f}s')

    writer.add_scalar('Train loss', error / len(train_dataloader), epoch)

    # Оценка качества на тестовых данных # This block should be indented
    model.eval()
    correct = 0
    with torch.no_grad():
        for x, y in test_dataloader:
            x, y = x.to(device), y.to(device)
            output = model(x)
            correct += (output.argmax(dim=1) == y).sum().item()

    accuracy = correct / len(test_dataset)
    writer.add_scalar('Test accuracy', accuracy, epoch)
    print(f'Accuracy: {accuracy:.4f}')

    # Сохранение лучшей модели
    if accuracy > best_accuracy:
        torch.save(model.state_dict(), 'best_mnist.pt')
        best_accuracy = accuracy

print('Training complete!')

Epoch 1/5, Loss: 1.4873, Time: 16.30s
Accuracy: 0.9721
Epoch 2/5, Loss: 1.4843, Time: 11.94s
Accuracy: 0.9720
Epoch 3/5, Loss: 1.4814, Time: 11.89s
Accuracy: 0.9736
Epoch 4/5, Loss: 1.4792, Time: 11.85s
Accuracy: 0.9737
Epoch 5/5, Loss: 1.4777, Time: 11.94s
Accuracy: 0.9729
Training complete!


In [9]:
# Загрузка и тестирование модели
model.load_state_dict(torch.load('best_mnist.pt'))
model.eval()

<ipython-input-9-923bb876e09a>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_mnist.pt'))


MNISTPerceptron(
  (linear1): Linear(in_features=784, out_features=200, bias=True)
  (linear2): Linear(in_features=200, out_features=10, bias=True)
)

In [10]:
# Пример предсказания
example = train_dataset[0][0].unsqueeze(0).to(device)
output = model(example)
print(f'Predicted label: {output.argmax(dim=1).item()}')


Predicted label: 5
